In [1]:
!pip install -qU datasets pandas pymongo sentence-transformers transformers
# Install below if using GPU
!pip install -qU accelerate bitsandbytes


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
# Load Dataset
from datasets import load_dataset
import pandas as pd

# https://huggingface.co/datasets/AIatMongoDB/embedded_movies
dataset = load_dataset("AIatMongoDB/embedded_movies")

# Convert the dataset to a pandas dataframe
dataset_df = pd.DataFrame(dataset["train"])

# Get length of df
print(f"Number of records in dataframe: {len(dataset_df)}")

Generating train split: 0 examples [00:00, ? examples/s]

Number of records in dataframe: 1500


In [3]:
# Data Preparation

# Remove data point where plot column is missing
dataset_df = dataset_df.dropna(subset=["fullplot"])
print(f"Number of records after dropping empty plots: {len(dataset_df)}")

# Remove the plot_embedding from each data point in the dataset as we are going to create new embeddings with the new OpenAI embedding Model "text-embedding-3-small"
dataset_df = dataset_df.drop(columns=["plot_embedding"])
dataset_df.head(1)

Number of records after dropping empty plots: 1452


,num_mflix_comments,title,type,poster,rated,metacritic,genres,directors,writers,awards,cast,plot,languages,imdb,countries,fullplot,runtime
0,0,The Perils of Pauline,movie,https://m.media-amazon.com/images/M/MV5BMzgxOD...,None,NaN,[Action],"[Louis J. Gasnier, Donald MacKenzie]","[Charles W. Goddard (screenplay), Basil Dickey...","{'nominations': 0, 'text': '1 win.', 'wins': 1}","[Pearl White, Crane Wilbur, Paul Panzer, Edwar...",Young Pauline is left a lot of money when her ...,[English],"{'id': 4465, 'rating': 7.6, 'votes': 744}",[USA],Young Pauline is left a lot of money when her ...,199.0


In [ ]:
from sentence_transformers import SentenceTransformer

# https://huggingface.co/thenlper/gte-large
embedding_model = SentenceTransformer("thenlper/gte-large")


def get_embedding(text: str) -> list[float]:
    if not text.strip():
        print("Attempted to get embedding for empty text.")
        return []

    embedding = embedding_model.encode(text)

    return embedding.tolist()


dataset_df["embedding"] = dataset_df["fullplot"].apply(get_embedding)

dataset_df.head(1)

modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.9k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/670M [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

KeyboardInterrupt: 

In [5]:
import pymongo
import getpass


def get_mongo_client(mongo_uri):
    """Establish connection to the MongoDB."""
    try:
        client = pymongo.MongoClient(mongo_uri)
        print("Connection to MongoDB successful")
        return client
    except pymongo.errors.ConnectionFailure as e:
        print(f"Connection failed: {e}")
        return None


mongo_uri = getpass.getpass("MONGO_URI")
if not mongo_uri:
    print("MONGO_URI not set in environment variables")

mongo_client = get_mongo_client(mongo_uri)

# Ingest data into MongoDB
db = mongo_client["movies"]
collection = db["movie_collection"]

In [21]:
# Delete any existing records in the collection
collection.delete_many({})

DeleteResult({'n': 0, 'electionId': ObjectId('7fffffff00000000000000f6'), 'opTime': {'ts': Timestamp(1710265561, 249), 't': 246}, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1710265561, 249), 'signature': {'hash': b'4\x9fw+\xa3\x01\xb4\x12?\x06~H\x1a\xd26\x96\x0b\xc6\xf2|', 'keyId': 7299545392000008318}}, 'operationTime': Timestamp(1710265561, 249)}, acknowledged=True)

In [22]:
documents = dataset_df.to_dict("records")
collection.insert_many(documents)

print("Data ingestion into MongoDB completed")

Data ingestion into MongoDB completed


In [23]:
def vector_search(user_query, collection):
    """
    Perform a vector search in the MongoDB collection based on the user query.

    Args:
    user_query (str): The user's query string.
    collection (MongoCollection): The MongoDB collection to search.

    Returns:
    list: A list of matching documents.
    """

    # Generate embedding for the user query
    query_embedding = get_embedding(user_query)

    if query_embedding is None:
        return "Invalid query or embedding generation failed."

    # Define the vector search pipeline
    pipeline = [
        {
            "$vectorSearch": {
                "index": "vector_index",
                "queryVector": query_embedding,
                "path": "embedding",
                "numCandidates": 150,  # Number of candidate matches to consider
                "limit": 4,  # Return top 4 matches
            }
        },
        {
            "$project": {
                "_id": 0,  # Exclude the _id field
                "fullplot": 1,  # Include the plot field
                "title": 1,  # Include the title field
                "genres": 1,  # Include the genres field
                "score": {"$meta": "vectorSearchScore"},  # Include the search score
            }
        },
    ]

    # Execute the search
    results = collection.aggregate(pipeline)
    return list(results)

In [24]:
def get_search_result(query, collection):

    get_knowledge = vector_search(query, collection)

    search_result = []
    for result in get_knowledge:
        search_result.append(
            {"title": result.get("title", "N/A"), "text": result.get("fullplot", "N/A")}
        )

    return search_result

In [26]:
# Conduct query with retrival of sources
query = "What is the best romantic movie to watch and why?"
source_information = get_search_result(query, collection)
combined_information = f"Query: {query}\nContinue to answer the query by using the Search Results:\n{source_information}."
print(combined_information)

{'title': 'Shut Up and Kiss Me!', 'text': "Ryan and Pete are 27-year old best friends in Miami, born on the same day and each searching for the perfect woman. Ryan is a rookie stockbroker living with his psychic Mom. Pete is a slick surfer dude yet to find commitment. Each meets the women of their dreams on the same day. Ryan knocks heads in an elevator with the gorgeous Jessica, passing out before getting her number. Pete falls for the insatiable Tiara, but Tiara's uncle is mob boss Vincent Bublione, charged with her protection. This high-energy romantic comedy asks to what extent will you go for true love?"}
--------
{'title': 'Pearl Harbor', 'text': 'Pearl Harbor is a classic tale of romance set during a war that complicates everything. It all starts when childhood friends Rafe and Danny become Army Air Corps pilots and meet Evelyn, a Navy nurse. Rafe falls head over heels and next thing you know Evelyn and Rafe are hooking up. Then Rafe volunteers to go fight in Britain and Evelyn 

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(load_in_4bit=True)

model_id = "CohereForAI/c4ai-command-r-v01"
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
# CPU Enabled uncomment below 👇🏽
# model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True)
# GPU Enabled use below 👇🏽
model = AutoModelForCausalLM.from_pretrained(
    model_id, trust_remote_code=True, quantization_config=bnb_config, device_map="auto"
)

In [ ]:
# Format message with the command-r chat template
messages = [{"role": "user", "content": combined_information}]
input_ids = tokenizer.apply_chat_template(
    messages, tokenize=True, add_generation_prompt=True, return_tensors="pt"
).to("cuda")

In [ ]:
gen_tokens = model.generate(
    input_ids,
    do_sample=True,
    temperature=0.3,
    max_new_tokens=100,
)

gen_text = tokenizer.decode(gen_tokens[0])
print(gen_text)